# Import Library

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras

# Load Dataset

In [2]:
data_path = "../Latihan-2-Bear/Dataset/"

images = []

for filename in os.listdir(data_path):
    img = tf.keras.preprocessing.image.load_img(f"{data_path}{filename}", target_size=(64, 64))
    img = tf.keras.preprocessing.image.img_to_array(img)
    images.append(img)

# Preprocessing

In [4]:
# Preprocessing

images = np.asarray(images)

# Shuffle

indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]

# Normalize
images = images.astype(np.float32) / 255.0

# Splitting Data
total_data = len(images)
train_count = int(total_data*0.8)

x_train = images[:train_count]
x_test = images[train_count:]

print (f"Total Data: {total_data}")
print (f"Train : {x_train.shape}, Test : {x_test.shape}")

Total Data: 300
Train : (240, 64, 64, 3), Test : (60, 64, 64, 3)


# Static Data

In [ ]:
BATCH_SIZE = 100
IMAGE_ORIGINAL_SHAPE = x_train.shape[1:]
IMAGE_SIZE = IMAGE_ORIGINAL_SHAPE[0] * IMAGE_ORIGINAL_SHAPE[1] * IMAGE_ORIGINAL_SHAPE[2]
HIDDEN_DIM = 512
LATENT_DIM = 32
EPOCHS = 60
LEARNING_RATE = 0.0001

# Define Model

In [ ]:
class VAE(tf.keras.Model):

    def __init__(self, dim, **kwargs):
        super(VAE, self).__init__(**kwargs)

        h_dim = dim[0]
        z_dim = dim[1]

        self.fc1 = tf.keras.layers.Dense(h_dim, activation=tf.nn.relu)
        self.fc2 = tf.keras.layers.Dense(z_dim)
        self.fc3 = tf.keras.layers.Dense(z_dim)

        self.fc4 = tf.keras.layers.Dense(h_dim, activation=tf.nn.relu)
        self.fc5 = tf.keras.layers.Dense(IMAGE_SIZE)

        self.encoder_normal_batch = tf.keras.layers.BatchNormalization()
        self.decoder_normal_batch = tf.keras.layers.BatchNormalization()

    def encode(self, x):
        x = self.encoder_normal_batch(x)
        h = self.fc1(x)
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = tf.exp(log_var * 0.5)
        eps = tf.random.normal(tf.shape(std))
        return mu + std * eps
    
    def decode_logits(self, z):
        h = self.fc4(z)
        h = self.fc5(h)
        return self.decoder_normal_batch(h)
    
    def decode(self, z):
        return tf.nn.sigmoid(self.decode_logits(z))

    def call(self, inputs, training=None, mask=None):
        mu, log_var = self.encode(inputs)
        z = self.reparameterize(mu, log_var)
        x_recon_logits = self.decode_logits(z)
        return x_recon_logits, mu, log_var

# Init Model

In [ ]:
model = VAE([HIDDEN_DIM, LATENT_DIM])
dummy = ((BATCH_SIZE, IMAGE_SIZE))

model(dummy)
model.summary()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.shuffle(BATCH_SIZE*5).batch(BATCH_SIZE)

# Train Model

# Plotting

# Show Original and Reconstruction Image

In [ ]:
mu, log_var = model.encode(x_test.reshape(-1, IMAGE_SIZE))
z = model(mu)